In [1]:
#!/usr/bin/env python
import os
import sys
import rospy
import rosbag
import copy
import tf
import cv2
from cv_bridge import CvBridge, CvBridgeError
import numpy as np
import std_msgs.msg
from std_msgs.msg import UInt16, String
import geometry_msgs.msg
from math import *
from numpy.linalg import inv
import scipy.io
from mat4py import loadmat
import actionlib
from robotiq_2f_gripper_msgs.msg import CommandRobotiqGripperFeedback, CommandRobotiqGripperResult, CommandRobotiqGripperAction, CommandRobotiqGripperGoal
from robotiq_2f_gripper_control.robotiq_2f_gripper_driver import Robotiq2FingerGripperDriver as Robotiq
from sensor_msgs.msg import Image, CameraInfo
import message_filters
from visualization_msgs.msg import Marker
from visualization_msgs.msg import MarkerArray
from geometry_msgs.msg import Point
import random
import urx
import math3d as m3d
import logging
import matplotlib.pyplot as plt
rospy.init_node('experiment_single_object', anonymous=True) #initialize the node



In [3]:
rospy.logwarn("Client test: Starting sending goals")
rob = urx.Robot("192.168.1.102")
logging.basicConfig(level=logging.WARN)
rob.set_tcp((0, 0, 0, 0, 0, 0))

[WARN] [1612531385.824602]: Client test: Starting sending goals


In [4]:
action_name = rospy.get_param('~action_name', 'command_robotiq_action')
robotiq_client = actionlib.SimpleActionClient(action_name, CommandRobotiqGripperAction)
robotiq_client.wait_for_server()

True

In [5]:
import serial
import signal
import time

serialPort = "/dev/Arduino"
baudRate = 57600
ser = serial.Serial(serialPort, baudRate, timeout=0.5)
def myHandler(signum, frame):
    pass

In [6]:
def go_to_home():
    
    Hong_joint0 = radians(101.23)
    Hong_joint1 = radians(-83.77)
    Hong_joint2 = radians(-130.28)
    Hong_joint3 = radians(-55.97)
    Hong_joint4 = radians(89.33)
    Hong_joint5 = radians(-78.73)
    
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.6, 1.5)
    
def go_to_safe_place():
    
    Hong_joint0 = radians(101.70)
    Hong_joint1 = radians(-75.56)
    Hong_joint2 = radians(-116.96)
    Hong_joint3 = radians(-77.49)
    Hong_joint4 = radians(89.42)
    Hong_joint5 = radians(-78.13)
    
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.6, 1.5)
    
def go_to_end_place():
    
    Hong_joint0 = radians(106.13)
    Hong_joint1 = radians(-60.48)
    Hong_joint2 = radians(-130.69)
    Hong_joint3 = radians(-78.86)
    Hong_joint4 = radians(89.68)
    Hong_joint5 = radians(-73.78)
    
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.6, 1.5)

In [7]:
from math import *
import numpy as np
def finger_tip_position_wrt_gripper_frame(alpha, l0, l1, l2l, l2r, flength_l, flength_r):
    alpha=alpha*pi/180
    left_fingertip_position=[-l0-l1*sin(alpha)+l2l, -l1*cos(alpha)-flength_l]
    right_fingertip_position=[l0+l1*sin(alpha)-l2r, -l1*cos(alpha)-flength_r]
    return left_fingertip_position, right_fingertip_position

def pos_flength_adjust_finger_fixed(current_alpha, next_alpha, theta, l0, l1, l2l, l2r, flength_r):
    theta=theta*pi/180
    current_flength_l = flength_r - (2*l0+2*l1*sin(current_alpha*pi/180)-l2l-l2r)/tan(theta)
    current_left_fingertip_pos_g, current_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(current_alpha, l0, l1, l2l, l2r, current_flength_l, flength_r)  
    next_left_fingertip_pos_g, next_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(next_alpha, l0, l1, l2l, l2r, current_flength_l, flength_r) 
    traslating_wst_g = [current_right_fingertip_pos_g[0]-next_right_fingertip_pos_g[0], current_right_fingertip_pos_g[1]-next_right_fingertip_pos_g[1]]
   # print np.array([[traslating_wst_g[0]], [traslating_wst_g[1]]])
    traslating_wst_world = np.array([[sin(theta), -cos(theta)], [cos(theta), sin(theta)]]).dot(np.array([[traslating_wst_g[0]], [traslating_wst_g[1]]])).ravel().tolist()
    next_left_fingertip_pos_g = [next_left_fingertip_pos_g[0]+traslating_wst_g[0], next_left_fingertip_pos_g[1]+traslating_wst_g[1]]
    next_right_fingertip_pos_g = [next_right_fingertip_pos_g[0]+traslating_wst_g[0], next_right_fingertip_pos_g[1]+traslating_wst_g[1]]
    extension_distance = -(next_right_fingertip_pos_g[1]+(2*l0+2*l1*sin(next_alpha*pi/180)-l2l-l2r)/tan(theta)-next_left_fingertip_pos_g[1])
    return traslating_wst_world, extension_distance

def pos_flength_adjust_thumb_fixed(current_alpha, next_alpha, theta, l0, l1, l2l, l2r, flength_r):
    theta=theta*pi/180
    current_flength_l = flength_r - (2*l0+2*l1*sin(current_alpha*pi/180)-l2l-l2r)/tan(theta)
    current_left_fingertip_pos_g, current_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(current_alpha, l0, l1, l2l, l2r, current_flength_l, flength_r)  
    next_left_fingertip_pos_g, next_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(next_alpha, l0, l1, l2l, l2r, current_flength_l, flength_r) 
    traslating_wst_g = [current_right_fingertip_pos_g[0]-next_right_fingertip_pos_g[0], current_right_fingertip_pos_g[1]-next_right_fingertip_pos_g[1]]
    traslating_wst_world = np.array([[sin(theta), -cos(theta)], [cos(theta), sin(theta)]]).dot(np.array([[traslating_wst_g[0]], [traslating_wst_g[1]]])).ravel().tolist()
    next_left_fingertip_pos_g = [next_left_fingertip_pos_g[0]+traslating_wst_g[0], next_left_fingertip_pos_g[1]+traslating_wst_g[1]]
    next_right_fingertip_pos_g = [next_right_fingertip_pos_g[0]+traslating_wst_g[0], next_right_fingertip_pos_g[1]+traslating_wst_g[1]]
    extension_distance = -(next_right_fingertip_pos_g[1]+(2*l0+2*l1*sin(next_alpha*pi/180)-l2l-l2r)/tan(theta)-next_left_fingertip_pos_g[1])
    traslating_wst_world[0]=traslating_wst_world[0]-((2*l0+2*l1*sin(current_alpha*pi/180)-l2l-l2r)-(2*l0+2*l1*sin(next_alpha*pi/180)-l2l-l2r))/sin(theta)
    return traslating_wst_world, extension_distance

#print pos_flength_adjust_thumb_fixed(10, 9, 60, 0.0125, 0.1, 0.006, 0.019, 0.125)

In [18]:
from geometry_msgs.msg import WrenchStamped
from std_msgs.msg import UInt16
import time

def skim_grasping():
    go_to_safe_place()
    #Robotiq.goto(robotiq_client, pos=2.5*0.14/48.0+1.12/48.0, speed=0.5, force=10, block=True) 
    Robotiq.goto(robotiq_client, pos=8*0.14/48.0+1.12/48.0, speed=0.5, force=10, block=True) 
    signal.signal(signal.SIGALRM, myHandler)
    signal.setitimer(signal.ITIMER_REAL, 0.005)
    ser.write('1000')
    signal.setitimer(signal.ITIMER_REAL, 0)
    time.sleep(0.2)    
    rob.set_tcp((0.022, 0, 0.355, 0, 0, 0))
    #rob.set_tcp((0.014, 0, 0.3400, 0, 0, 0))  
    rospy.sleep(0.5)
    world2tcp = rob.get_pose()
    world2tcp.pos.x+=0.007
    world2tcp.pos.y-=0.001
    rob.translate((0.01-world2tcp.pos.x,0.675-world2tcp.pos.y,0), acc=0.5, vel=0.8, wait=True) 
    rob.translate((0, 0, -0.106), acc=0.5, vel=0.8) #0.116  #-0.104
    rob.movel_tool((0,0,0,0,0,-pi/2), acc=0.5, vel=0.8, wait=True)
    rob.translate_tool((0.006, 0, 0), acc=0.06, vel=0.08)     #0.006
    
    rob.movel_tool((0,0,0,0,45*pi/180,0), acc=0.3, vel=0.8, wait=True)    
    
    # Create ros bag
    #bag = rosbag.Bag('./torque_y_finger30.bag', 'w')
    
    rob.translate((0, 0, -0.02), acc=0.01, vel=0.05, wait=False)
    wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
    ini_wrench = wrench.wrench.torque.y
    time0 = time.time()
    num_large_force = 0
    time.sleep(0.2)
    while True:
        # Record torque and save to ros bag
        #bag.write('torque_y', wrench)
        
        if num_large_force == 3:
            rob.stopl()
            break
        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
        if wrench.wrench.torque.y>ini_wrench+0.4:   
            num_large_force += 1
        if time.time()-time0>2.8:  
            break
    time.sleep(0.5)
    '''
    time1 = time.time()
    while True:
        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
        bag.write('torque_y', wrench)
        if time.time()-time1>0.5:
            break 
    bag.close()
    '''
    
    #bag = rosbag.Bag('./torque_y_finger31.bag', 'w')
    wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
    ini_wrench = wrench.wrench.torque.y
    for current_finger_length_extend in np.arange(0,70,4):
        current_finger_length = 0+current_finger_length_extend #Change
        signal.setitimer(signal.ITIMER_REAL, 0.001)        
        ser.write('1'+str(current_finger_length))
        signal.setitimer(signal.ITIMER_REAL, 0)
        time.sleep(0.2)        
        current_fingerlength_int=current_finger_length
        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
        #bag.write('torque_y', wrench)
        if wrench.wrench.torque.y>ini_wrench+0.3:
            break
    print current_fingerlength_int 
    time.sleep(0.5)
    '''
    time1 = time.time()
    while True:
        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
        bag.write('torque_y', wrench)
        if time.time()-time1>0.5:
            break
    bag.close()
    '''
    
    angle_step=0.2
    for angle in np.arange(8,-4,-angle_step):
        translate_dir_dis, extension_distance = pos_flength_adjust_thumb_fixed(angle, angle-angle_step, 60, 0.0125, 0.1, 0.019, 0.01, 0.125)
        old_fingerlength_int=current_fingerlength_int
        current_fingerlength_int=round(max(current_fingerlength_int+extension_distance*(180.0/0.03),0))
        signal.signal(signal.SIGALRM, myHandler)
        signal.setitimer(signal.ITIMER_REAL, 0.005)
        ser.write('1'+str(current_fingerlength_int))
        signal.setitimer(signal.ITIMER_REAL, 0)   
        #if abs(current_fingerlength_int-old_fingerlength_int)!=0 and current_fingerlength_int<=155:
        if current_fingerlength_int<=180:
            Robotiq.goto(robotiq_client, pos=(angle-1)*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
            rob.translate_tool((sin(60*pi/180)*translate_dir_dis[0]+cos(60*pi/180)*translate_dir_dis[1], 0, cos(60*pi/180)*translate_dir_dis[0]-sin(60*pi/180)*translate_dir_dis[1]), acc=0.05, vel=0.05, wait=True)
        #else:
            #time.sleep(0.01)
    Robotiq.goto(robotiq_client, pos=-4*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
    rospy.sleep(0.5)
    rob.translate_tool((0, 0, -0.12), acc=0.3, vel=0.8, wait=True)
    rob.movel_tool((0,0,0,0,-pi/4,0), acc=0.5, vel=0.8, wait=True)
    rospy.sleep(0.5)
    go_to_end_place()
    Robotiq.goto(robotiq_client, pos=2*0.14/48.0+1.12/48.0, speed=0.5, force=10, block=True)  


In [19]:
#bag_number = 0
for i in range(30):
    #print i0
    skim_grasping()
#    bag_number = bag_number + 1

68
68
68
68
68
68
68
68
60
56


KeyboardInterrupt: 

In [ ]:
signal.setitimer(signal.ITIMER_REAL, 0.001)
ser.write('1000')
signal.setitimer(signal.ITIMER_REAL, 0)
time.sleep(1)


In [20]:
Robotiq.goto(robotiq_client, pos=40*0.14/48.0+1.12/48.0, speed=0.1, force=10, block=True)

In [ ]:
softtip()

In [154]:
go_to_safe_place()

In [ ]:
rob.set_tcp((-0.021, 0, 0.330, 0, 0, 0))  
rob.movel_tool((0,0,0,0,20*pi/180,0), acc=0.3, vel=0.1, wait=True)    
rospy.sleep(0.5)
rob.movel_tool((0,0,0,0,-20*pi/180,0), acc=0.3, vel=0.1, wait=True)    

In [ ]:
rob.translate_tool((0, 0, 0.02), acc=0.01, vel=0.05, wait=False)

In [33]:
from geometry_msgs.msg import WrenchStamped
from std_msgs.msg import UInt16
import time
Hong_joint0 = radians(98.77)
Hong_joint1 = radians(-92.59)
Hong_joint2 = radians(-101.78)
Hong_joint3 = radians(-75.63)
Hong_joint4 = radians(89.10)
Hong_joint5 = radians(-171.05)
rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.5,0.8)
Robotiq.goto(robotiq_client, pos=13.2*0.14/48.0+1.12/48.0, speed=0.1, force=10, block=True)
signal.setitimer(signal.ITIMER_REAL, 0.001)
ser.write('1100')
signal.setitimer(signal.ITIMER_REAL, 0)
time.sleep(1)
rob.set_tcp((-0.021, 0, 0.330, 0, 0, 0))  
rospy.sleep(0.5)
rob.movel_tool((0,0,0,0,45*pi/180,0), acc=0.5, vel=0.8, wait=True) 

rob.translate((-0.11,0,0), acc=0.5, vel=0.8, wait=True) 
rob.translate((0,0,-0.13), acc=0.5, vel=0.8, wait=True) 

rob.translate((0, 0, -0.02), acc=0.01, vel=0.05, wait=False)
wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
ini_wrench = wrench.wrench.torque.y
time0 = time.time()
num_large_force = 0
time.sleep(0.2)
while True:
        # Record torque and save to ros bag
        #bag.write('torque_y', wrench)
        
    if num_large_force == 3:
        rob.stopl()
        break
    wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
    if wrench.wrench.torque.y>ini_wrench+0.04:   
        num_large_force += 1
    if time.time()-time0>2.8:  
        break
time.sleep(0.5)
rob.translate((-0.054,0,0,0,0,0), acc=0.01, vel=0.05, wait=True) 
rob.set_tcp((0, 0, 0, 0, 0, 0))  
rospy.sleep(0.5)
